In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [3]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [4]:
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F

In [5]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [6]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 135.,  57.],
        [ 88., 134.,  59.],
        [ 68.,  97.,  70.],
        [ 87., 134.,  58.],
        [101.,  44.,  37.]])
tensor([[118., 134.],
        [118., 132.],
        [102., 120.],
        [119., 133.],
        [ 21.,  38.]])


In [7]:
model = nn.Linear(3, 2)
list(model.parameters())

[Parameter containing:
 tensor([[-0.1994,  0.5607, -0.2439],
         [-0.4542,  0.0166, -0.4031]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4005, -0.3297], requires_grad=True)]

In [8]:
preds = model(inputs)
preds

tensor([[ 12.9241, -49.7079],
        [ 15.9876, -66.0005],
        [ 44.0398, -61.0037],
        [ -4.8502, -60.8571],
        [ 23.3956, -58.2952],
        [ 12.1641, -50.1786],
        [ 15.1831, -66.4201],
        [ 43.5965, -61.8610],
        [ -4.0902, -60.3864],
        [ 23.3511, -58.2442],
        [ 12.1195, -50.1276],
        [ 15.2276, -66.4712],
        [ 44.8444, -60.5840],
        [ -4.8057, -60.9082],
        [ 24.1556, -57.8245]], grad_fn=<AddmmBackward0>)

In [9]:
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(13969.0059, grad_fn=<MseLossBackward0>)


In [15]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [18]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [19]:
fit(200, model, loss_fn, opt, train_dl)

Epoch [10/200], Loss: 33.3756
Epoch [20/200], Loss: 23.0087
Epoch [30/200], Loss: 19.9386
Epoch [40/200], Loss: 14.3260
Epoch [50/200], Loss: 21.6584
Epoch [60/200], Loss: 19.7503
Epoch [70/200], Loss: 15.1035
Epoch [80/200], Loss: 12.4008
Epoch [90/200], Loss: 21.5261
Epoch [100/200], Loss: 11.8191
Epoch [110/200], Loss: 13.0743
Epoch [120/200], Loss: 14.6322
Epoch [130/200], Loss: 12.9951
Epoch [140/200], Loss: 11.9700
Epoch [150/200], Loss: 7.7557
Epoch [160/200], Loss: 13.6707
Epoch [170/200], Loss: 10.3528
Epoch [180/200], Loss: 12.8146
Epoch [190/200], Loss: 11.3756
Epoch [200/200], Loss: 8.6043


In [20]:
model(inputs)

tensor([[ 57.1893,  70.4864],
        [ 80.3354,  98.7339],
        [121.5873, 136.4130],
        [ 21.6904,  38.6781],
        [ 98.4849, 114.7836],
        [ 55.9043,  69.3728],
        [ 79.8867,  98.5055],
        [121.7102, 136.8406],
        [ 22.9754,  39.7917],
        [ 99.3210, 115.6687],
        [ 56.7405,  70.2579],
        [ 79.0505,  97.6203],
        [122.0361, 136.6414],
        [ 20.8543,  37.7929],
        [ 99.7698, 115.8972]], grad_fn=<AddmmBackward0>)

In [21]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])